In [2]:
%pylab inline
import math
import numpy as np
import scipy as sp
from scipy import stats
import pylab as pb
import GPy
import matplotlib.pyplot as plt
from IPython.core.debugger import Tracer

Populating the interactive namespace from numpy and matplotlib


In [ ]:
psa_pop = {str(i[0]): i[1] for i in np.genfromtxt('../data/info/PSA_processed.csv', delimiter=',')}
weekly_theft_count = {str(i[0]): i[1] for i in np.genfromtxt('../data/processed/train/weekonly_theft.csv', delimiter=',')}
train_theft = np.genfromtxt('../data/processed/train/week_theft.csv', delimiter=',')
train_theft_dict = {str(i[1])+str(i[0]): i[2] for i in train_theft}
# Training input
time_train =train_theft[:, 0].ravel()
space_train = train_theft[:, 1].ravel()
space_grid, time_grid = np.meshgrid(np.unique(space_train), np.unique(time_train))
X_train = np.vstack([space_grid.ravel(), time_grid.ravel()]).T
# Training output
intensity_train = np.array([[train_theft_dict.get('{}{}'.format(*i), 0) for i in X_train]]).T
log_es = np.array([[math.log(weekly_theft_count[str(i[1])] * psa_pop[str(i[0])]) for i in X_train]]).T
truth_lamda = np.exp(intensity_train + log_es)
truth_lamda_grid = truth_lamda.reshape(len(weekly_theft_count.keys()), len(psa_pop.keys()))
Y_train = np.array([[np.random.poisson(i) for i in truth_lamda]]).T

In [ ]:
# Make a kernel for the space_time effect, f
kern_f = GPy.kern.RBF(2, active_dims=[0,1], name='space_time_kern')
# Make a kernel for the spatial only effect (which is in log space)
kern_s_effect = GPy.kern.RBF(1, active_dims=[0], name='space_effect')

In [ ]:
t_distribution = GPy.likelihoods.StudentT(deg_free=4, sigma2=1)
laplace_inf = GPy.inference.latent_function_inference.Laplace()
full_kern = kern_f + kern_s_effect
m = GPy.core.GP(X=X_train, Y=Y_train, kernel=full_kern, likelihood=t_distribution, inference_method=inference_metho)

In [ ]:
print(m)